In [134]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl
import datetime as dt
import copy
pd.set_option('display.max_columns', None)


# Data

This section focuses on pulling the data and prepping/aggregating the dependent variable. (Fantasy Points)

In [7]:
roster_data = nfl.import_seasonal_rosters([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999])
pbp_df = pd.DataFrame(nfl.import_pbp_data([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))
weekly_df = pd.DataFrame(nfl.import_weekly_data([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))
# injuries_df = pd.DataFrame(nfl.import_injuries([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000]))
schedules_df = pd.DataFrame(nfl.import_schedules([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))

2024 done.
2023 done.
2022 done.
2021 done.
2020 done.
2019 done.
2018 done.
2017 done.
2016 done.
2015 done.
2014 done.
2013 done.
2012 done.
2011 done.
2010 done.
2009 done.
2008 done.
2007 done.
2006 done.
2005 done.
2004 done.
2003 done.
2002 done.
2001 done.
2000 done.
1999 done.
Downcasting floats.
Downcasting floats.


In [145]:
pbp_df[pbp_df['kicker_player_name'].notnull()]['field_goal_result'].unique()

array([None, 'made', 'missed', 'blocked'], dtype=object)

In [148]:
def get_opposing_team(df):
    if df['home_team'] == df['team']:
        val = df['away_team']
    else:
        val = df['home_team']

    return val


kicker_plays = pbp_df[pbp_df['kicker_player_name'].notnull()]

# kicker_plays['kicking_against_team'] = kicker_plays.apply(get_kicker_opposition,axis = 1)

# Filter out only successful field goals for longest field goal calculation

made_field_goals = kicker_plays[kicker_plays['field_goal_result'] == 'made']


## <40

kicker_plays['sub_40_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance'] < 40) , 1, 0), axis = 1)
kicker_plays['sub_40_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] != None) & (row['kick_distance'] < 40) , 1, 0), axis = 1)

## 40-49

kicker_plays['40_49_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance']  >= 40)  & (row['kick_distance'] < 50), 1, 0), axis = 1)
kicker_plays['40_49_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result']!= None) & (row['kick_distance']  >= 40)  & (row['kick_distance'] < 50), 1, 0), axis = 1)


## 50+ 

kicker_plays['50_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance']  >= 50), 1, 0), axis = 1)
kicker_plays['50_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result']!= None) & (row['kick_distance']  >= 50), 1, 0), axis = 1)




kicker_stats = kicker_plays.groupby(['game_id', 'game_date', 'week', 'season','div_game', 'home_team', 'away_team', 'weather', 'location', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach','kicker_player_id', 'kicker_player_name']).agg({
    'field_goal_result': lambda x: (x == 'made').sum(),  # Sum of successful field goals
    'extra_point_result': lambda x: (x == 'good').sum(),  # Sum of successful extra points
    'kick_distance': 'sum',  # Sum of kickoff yardage
    'field_goal_attempt': 'sum',  # Total field goal attempts
    'kickoff_attempt': 'sum',
    'extra_point_attempt' : 'sum',
    'sub_40_fg_make':'sum',
    'sub_40_fg_attempt':'sum',
    '40_49_fg_make':'sum',
    '40_49_fg_attempt':'sum',
    '50_fg_make':'sum',
    '50_fg_attempt' : 'sum'

}).reset_index()



C:\Users\chris\AppData\Local\Temp\ipykernel_25528\416655711.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kicker_plays['sub_40_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance'] < 40) , 1, 0), axis = 1)
C:\Users\chris\AppData\Local\Temp\ipykernel_25528\416655711.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kicker_plays['sub_40_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] != None) & (row['kick_dis

**Aggregating Kicking related features and calculating Dependent Variable**

In [213]:

# 5 pts per 50+ yard FG made
# 4 pts per 40-49 yard FG made
# 3 pts per FG made, 39 yards or less

def get_opposing_team(df):
    if df['home_team'] == df['team']:
        val = df['away_team']
    elif df['away_team'] == df['team']:
        val = df['home_team']
    else:
        val = None

    return val


kicker_plays = pbp_df[pbp_df['kicker_player_name'].notnull()]

# kicker_plays['kicking_against_team'] = kicker_plays.apply(get_kicker_opposition,axis = 1)

# Filter out only successful field goals for longest field goal calculation

made_field_goals = kicker_plays[kicker_plays['field_goal_result'] == 'made']


## <40

kicker_plays['sub_40_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance'] < 40) , 1, 0), axis = 1)
kicker_plays['sub_40_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] != None) & (row['kick_distance'] < 40) , 1, 0), axis = 1)

## 40-49

kicker_plays['40_49_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance']  >= 40)  & (row['kick_distance'] < 50), 1, 0), axis = 1)
kicker_plays['40_49_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result']!= None) & (row['kick_distance']  >= 40)  & (row['kick_distance'] < 50), 1, 0), axis = 1)


## 50+ 

kicker_plays['50_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance']  >= 50), 1, 0), axis = 1)
kicker_plays['50_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result']!= None) & (row['kick_distance']  >= 50), 1, 0), axis = 1)





kicker_stats = kicker_plays.groupby(['game_id', 'game_date', 'week', 'season','div_game', 'home_team', 'away_team', 'weather', 'location', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach','kicker_player_id', 'kicker_player_name']).agg({
    'field_goal_result': lambda x: (x == 'made').sum(),  # Sum of successful field goals
    'extra_point_result': lambda x: (x == 'good').sum(),  # Sum of successful extra points
    'kick_distance': 'sum',  # Sum of kickoff yardage
    'field_goal_attempt': 'sum',  # Total field goal attempts
    'kickoff_attempt': 'sum',
    'extra_point_attempt' : 'sum',
    'sub_40_fg_make':'sum',
    'sub_40_fg_attempt':'sum',
    '40_49_fg_make':'sum',
    '40_49_fg_attempt':'sum',
    '50_fg_make':'sum',
    '50_fg_attempt' : 'sum'

}).reset_index()




## Adding in the opponent team for the kicker of interest 

kicker_team = roster_data[['season','player_id','team','depth_chart_position']]

kicker_team.rename(columns = {'player_id':'kicker_player_id'},inplace = True)

kicker_stats = kicker_stats.merge(kicker_team, on = ['kicker_player_id','season'], how = 'inner')


# Calculate longest successful field goal using the filtered made_field_goals DataFrame
longest_fg = made_field_goals.groupby(['game_id','kicker_player_id', 'kicker_player_name'])['kick_distance'].max().reset_index()
longest_fg.rename(columns={'kick_distance': 'longest_fg'}, inplace=True)
# Merge longest field goal into kicker_stats
kicker_stats = pd.merge(kicker_stats, longest_fg, on=['game_id', 'kicker_player_id', 'kicker_player_name'], how='left')

# Rename the aggregated columns for clarity
kicker_stats.rename(columns={
    'field_goal_attempt': 'fg_attempts',
    'field_goal_result': 'fg_made',
    'longest_fg': 'longest_fg',
    'extra_point_result': 'xp_made',
    'extra_point_attempt': 'xp_attempts',
    'kick_distance': 'total_kick_distance'
}, inplace=True)

kicker_stats.drop(columns=['kicker_player_name'], inplace=True)

## Aggregate average score to opposition 

kicker_stats['opponent_team'] = kicker_stats.apply(get_opposing_team,axis = 1)



home_teams = schedules_df[['season','home_team','away_score']].copy()

away_teams = schedules_df[['season','away_team','home_score']].copy()

home_teams.rename(columns = {'home_team':'team','away_score':'points_allowed'}, inplace = True)
away_teams.rename(columns = {'away_team':'team','home_score':'points_allowed'}, inplace = True)

points_allowed_df = pd.concat([home_teams,away_teams])


points_allowed_df['avg_points_allowed'] = points_allowed_df.groupby(['season','team'])['points_allowed'].transform('mean')


points_allowed_df.rename(columns = {'team':'opponent_team'},inplace = True)


kicker_stats = kicker_stats.merge(points_allowed_df[['season','opponent_team','avg_points_allowed']].drop_duplicates(), on = ['opponent_team','season'], how = 'left')

## Creating Average Field Goals Per Game

kicker_stats['average_field_goals'] = kicker_stats.groupby(['kicker_player_id','season'])['fg_made'].transform('mean')

## Creating Field Goal Make Percentage (Seasonal)

kicker_stats['total_made'] = kicker_stats.groupby(['kicker_player_id','season'])['fg_made'].transform('sum')
kicker_stats['total_attempts'] = kicker_stats.groupby(['kicker_player_id','season'])['fg_attempts'].transform('sum')

kicker_stats['season_make_percentage_fg'] = round(kicker_stats['total_made'].astype(float) / kicker_stats['total_attempts'].astype(float),2)


## Creating Average Extra Points Per Game


kicker_stats['average_xp'] = kicker_stats.groupby(['kicker_player_id','season'])['xp_made'].transform('mean')

## Creating Extra Point Make Percentage (Seasonal)

kicker_stats['total_made_xp'] = kicker_stats.groupby(['kicker_player_id','season'])['xp_made'].transform('sum')
kicker_stats['total_attempts_xp'] = kicker_stats.groupby(['kicker_player_id','season'])['xp_attempts'].transform('sum')

kicker_stats['season_make_percentage-xp'] = round(kicker_stats['total_made_xp'].astype(float) / kicker_stats['total_attempts_xp'].astype(float),2)

for_fantasy_scoring = pbp_df[pbp_df['kicker_player_name'].notnull()]



## Creating Average Extra Point Attempts (Proxy for how often Team Scores)


kicker_stats['xp_attempts_per_game'] = round(kicker_stats.groupby(['kicker_player_id','season'])['xp_attempts'].transform('mean'),2)


## Average Fantasy Points Allowed To Kickers

## scoring criteria 
# https://www.espn.com/fantasy/football/ffl/story?page=fflrulesstandardscoring

kicker_stats['total_kicking_points_fantasy'] = (kicker_stats['50_fg_make'] * 5) + (kicker_stats['40_49_fg_make'] * 4) + (kicker_stats['sub_40_fg_make'] * 3) +((kicker_stats['40_49_fg_attempt'] - kicker_stats['40_49_fg_make']) * -1) + ((kicker_stats['sub_40_fg_attempt'] - kicker_stats['sub_40_fg_make']) * -2) + (kicker_stats['xp_made'] * 1)

# .apply(lambda row: (int(row['50_fg_make']) * 5) + (row['40_49_fg_make'] * 4) + (row['sub_40_fg_make'] * 3) + ((row['fg_attempts'] - row['fg_made']) * -1)) 


kicker_stats['opp_avg_kick_points_allowed'] = kicker_stats.groupby('opponent_team')['total_kicking_points_fantasy'].transform('mean')
 
# kicker_stats['points_from_kicks'] = kicker_stats['fg_made']
# kicker_stats['points_from_xp'] = kick


C:\Users\chris\AppData\Local\Temp\ipykernel_25528\3800031028.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kicker_plays['sub_40_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance'] < 40) , 1, 0), axis = 1)
C:\Users\chris\AppData\Local\Temp\ipykernel_25528\3800031028.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kicker_plays['sub_40_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] != None) & (row['kick_d

In [214]:

kicker_stats.shape


(10695, 47)

In [217]:
kicker_stats.head(2)

,game_id,game_date,week,season,div_game,home_team,away_team,weather,location,stadium,spread_line,total_line,roof,surface,temp,wind,home_coach,away_coach,kicker_player_id,fg_made,xp_made,total_kick_distance,fg_attempts,kickoff_attempt,xp_attempts,sub_40_fg_make,sub_40_fg_attempt,40_49_fg_make,40_49_fg_attempt,50_fg_make,50_fg_attempt,team,depth_chart_position,longest_fg,opponent_team,avg_points_allowed,average_field_goals,total_made,total_attempts,season_make_percentage_fg,average_xp,total_made_xp,total_attempts_xp,season_make_percentage-xp,xp_attempts_per_game,total_kicking_points_fantasy,opp_avg_kick_points_allowed
0,2001_01_ATL_SF,2001-09-09,1,2001,1,SF,ATL,"partly cloudy Temp: 68° F, Humidity: 63%, Wind...",Home,3COM Park,3.5,46.0,outdoors,grass,68.0,12.0,Steve Mariucci,Dan Reeves,00-0003501,3,1,333.0,3.0,4.0,1.0,3,3,0,0,0,0,SF,K,39.0,ATL,23.5625,1.076923,14,22.0,0.64,2.538462,33,33.0,1.0,2.54,10,5.966887
1,2001_02_STL_SF,2001-09-23,2,2001,1,SF,LA,"Partly Cloudy Temp: 59° F, Humidity: 74%, Wind...",Home,3COM Park,-6.0,50.0,outdoors,grass,59.0,10.0,Steve Mariucci,Mike Martz,00-0003501,2,2,460.0,2.0,5.0,2.0,1,1,0,0,1,1,SF,K,52.0,LA,NaN,1.076923,14,22.0,0.64,2.538462,33,33.0,1.0,2.54,10,6.705882


# Begin Modeling